## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-30-16-50-48 +0000,nyt,"Overlooked No More: Tina Modotti, Whose Life W...",https://www.nytimes.com/2025/08/30/obituaries/...
1,2025-08-30-16-50-25 +0000,nyt,3 Dead in Indonesia After Jumping From Buildin...,https://www.nytimes.com/2025/08/30/world/asia/...
2,2025-08-30-16-49-00 +0000,nyt,U.S. Is Working With Guatemala to Return Hundr...,https://www.nytimes.com/2025/08/29/us/immigrat...
3,2025-08-30-16-41-58 +0000,bbc,Prominent Ukrainian politician Andriy Parubiy ...,https://www.bbc.com/news/articles/cjw6ep37469o...
4,2025-08-30-16-33-41 +0000,latimes,'Moment of crisis': Unions in somber mood thi...,https://www.latimes.com/business/story/2025-08...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2344/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
84,trump,46
340,court,13
205,president,10
62,israeli,9
187,tariffs,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
119,2025-08-30-04-15-00 +0000,wsj,"A federal judge in Washington, D.C., temporari...",https://www.wsj.com/politics/policy/judge-bloc...,162
150,2025-08-29-23-59-00 +0000,wsj,A federal appeals court struck down the Trump ...,https://www.wsj.com/politics/policy/appeals-co...,113
175,2025-08-29-22-04-53 +0000,nypost,Trump’s tariffs ruled illegal in stunning blow...,https://nypost.com/2025/08/29/us-news/trumps-t...,109
228,2025-08-29-19-45-00 +0000,wsj,A federal judge signaled that she will move qu...,https://www.wsj.com/politics/policy/judge-weig...,102
172,2025-08-29-22-22-47 +0000,cbc,U.S. appeals court finds many Trump tariffs un...,https://www.cbc.ca/news/world/appeals-court-tr...,100


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
119,162,2025-08-30-04-15-00 +0000,wsj,"A federal judge in Washington, D.C., temporari...",https://www.wsj.com/politics/policy/judge-bloc...
228,39,2025-08-29-19-45-00 +0000,wsj,A federal judge signaled that she will move qu...,https://www.wsj.com/politics/policy/judge-weig...
150,38,2025-08-29-23-59-00 +0000,wsj,A federal appeals court struck down the Trump ...,https://www.wsj.com/politics/policy/appeals-co...
11,37,2025-08-30-16-05-58 +0000,nyt,Houthis Say Prime Minister Was Killed in Israe...,https://www.nytimes.com/2025/08/30/world/middl...
135,37,2025-08-30-01-00-00 +0000,wsj,Minnesota Gov. Tim Walz campaigned on the nati...,https://www.wsj.com/politics/elections/we-aske...
109,30,2025-08-30-07-22-55 +0000,bbc,US blocks Palestinian leader from attending UN...,https://www.bbc.com/news/articles/cjdym32z9v7o...
238,30,2025-08-29-19-21-21 +0000,nyt,Tesla’s Lawyers File Motion to Throw Out $243 ...,https://www.nytimes.com/2025/08/29/business/te...
138,30,2025-08-30-00-45-55 +0000,latimes,Former California city councilman molested und...,https://www.latimes.com/california/story/2025-...
25,29,2025-08-30-15-09-16 +0000,nyt,"Large Russian Air Attack Hits Ukraine, Amid St...",https://www.nytimes.com/2025/08/30/world/europ...
229,28,2025-08-29-19-42-00 +0000,wsj,Republican Sen. Joni Ernst isn’t expected to s...,https://www.wsj.com/politics/elections/republi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
